In [1]:
import numpy as np
from scipy.linalg import hadamard
from scipy.stats import bernoulli

In [5]:
def Bi(zi_mins, zi_maxs, k):
    si = zi_maxs - zi_mins
    arr = np.arange(k)*(si/(k-1))
    return zi_mins + arr

def encoder(zi, bi, iters, d, k):
    brs = np.zeros((iters, d, 2))
    
    for i in range(iters):
        ids = np.searchsorted(bi[i], zi[i], side='right')-1
        idid = np.vstack((ids, ids+1)).T
        idid[idid==k] = k-1
        brs[i] = bi[i][idid]
        
    probs = np.where(
        ((brs[..., 1] - brs[..., 0]) != 0), 
        np.divide((zi - brs[..., 0]),(brs[..., 1]- brs[..., 0] + (1e-100))), 
        0
    )
    
    return bernoulli.rvs(probs), brs

def decoder(enc, brs):
    return np.where(
        enc==1,  brs[..., 1], brs[..., 0]
    )

In [18]:
def fun(n, d, k, ITERS):
    X = np.random.rand(n, ITERS, d)
    
    R = hadamard(d) @ np.diag(
        np.random.choice([1, -1], size=d, p=[0.5, 0.5])
    ) / np.sqrt(d)
    
    assert np.allclose(np.eye(d), R @ R.T), "R is not orthogonal."
    
    Y = np.zeros((ITERS, d))
    tot_expec = 0
    
    for xi in X:
        zi = np.transpose(R @ xi.T)
        assert np.allclose(
            np.linalg.norm(zi),
            np.linalg.norm(xi)
        ), "l2 norm zi != l2 norm xi"
        
        zi_maxs = np.max(zi, axis=1, keepdims=True)
        zi_mins = np.min(zi, axis=1, keepdims=True)

        expec_zmin = np.mean(zi_mins**2)
        expec_zmax = np.mean(zi_maxs**2)
        up_bnd_minmax = (np.linalg.norm(
            xi[np.random.randint(ITERS)])**2/d) * ((np.log(d**2) + 2))

        print("expected_val_sq_zmax:\t\t   ", expec_zmax)
        print("expected_val_sq_zmin:\t\t   ", expec_zmin)
        print("upper bound for sq_zmax or sq_zmin:", up_bnd_minmax)
        print("-----------------------------------")
        bi = Bi(zi_maxs=zi_maxs, zi_mins=zi_mins, k=k)

        encs, brs = encoder(zi=zi, bi=bi, k=k,
                            iters=ITERS, d=d)

        yi = decoder(encs, brs)


        Y += yi
        tot_expec += expec_zmin+expec_zmax
    Z_hat_mean = Y/n 
    x_hat_mean = np.transpose(np.linalg.inv(R) @ Z_hat_mean.T)
    x_mean = np.mean(X, axis=0)

    error_obs = np.mean((np.linalg.norm((x_mean - x_hat_mean), axis=1)**2))
    error_cal_bnd = (d/(2*((n*(k-1))**2))) * tot_expec
    print("\n\nexpected error:\t\t", error_obs)
    print("upper bound for error:\t", error_cal_bnd)

In [22]:
fun(n=8, d=64, ITERS=1024, k=4)

expected_val_sq_zmax:		    2.0238600079481763
expected_val_sq_zmin:		    1.8550234501008604
upper bound for sq_zmax or sq_zmin: 3.866246690169994
-----------------------------------
expected_val_sq_zmax:		    2.03156085575089
expected_val_sq_zmin:		    1.8436387281061517
upper bound for sq_zmax or sq_zmin: 3.3402675640777924
-----------------------------------
expected_val_sq_zmax:		    2.038278008243674
expected_val_sq_zmin:		    1.8523194913471492
upper bound for sq_zmax or sq_zmin: 3.4291717943752893
-----------------------------------
expected_val_sq_zmax:		    2.041125117982749
expected_val_sq_zmin:		    1.8280133473504552
upper bound for sq_zmax or sq_zmin: 3.642384669526084
-----------------------------------
expected_val_sq_zmax:		    2.0714529884569917
expected_val_sq_zmin:		    1.8728262532585924
upper bound for sq_zmax or sq_zmin: 3.4729237514833042
-----------------------------------
expected_val_sq_zmax:		    2.040728737084788
expected_val_sq_zmin:		    1.883422169122721
u

In [26]:
fun(n=8, d=512, ITERS=4096, k=16)

expected_val_sq_zmax:		    3.7511226764765047
expected_val_sq_zmin:		    3.1898810581080648
upper bound for sq_zmax or sq_zmin: 4.643051702285406
-----------------------------------
expected_val_sq_zmax:		    3.73917376443236
expected_val_sq_zmin:		    3.172126083108739
upper bound for sq_zmax or sq_zmin: 4.583813236358468
-----------------------------------
expected_val_sq_zmax:		    3.74860390070347
expected_val_sq_zmin:		    3.171175196939717
upper bound for sq_zmax or sq_zmin: 4.639199228867067
-----------------------------------
expected_val_sq_zmax:		    3.7558228842128876
expected_val_sq_zmin:		    3.1599741128852648
upper bound for sq_zmax or sq_zmin: 4.753346468706009
-----------------------------------
expected_val_sq_zmax:		    3.7505121841501525
expected_val_sq_zmin:		    3.1662405991428404
upper bound for sq_zmax or sq_zmin: 4.7243523583257145
-----------------------------------
expected_val_sq_zmax:		    3.7471395332793858
expected_val_sq_zmin:		    3.1528655320932915
upp

In [29]:
fun(n=16, d=512, ITERS=8192, k=16)

expected_val_sq_zmax:		    2.8284385428757335
expected_val_sq_zmin:		    2.75691380451301
upper bound for sq_zmax or sq_zmin: 5.105284757423369
-----------------------------------
expected_val_sq_zmax:		    2.830549966705787
expected_val_sq_zmin:		    2.77633467126836
upper bound for sq_zmax or sq_zmin: 4.9504873890212995
-----------------------------------
expected_val_sq_zmax:		    2.836548955151663
expected_val_sq_zmin:		    2.770661243803754
upper bound for sq_zmax or sq_zmin: 4.79556990893662
-----------------------------------
expected_val_sq_zmax:		    2.8394462229801425
expected_val_sq_zmin:		    2.7585510641461575
upper bound for sq_zmax or sq_zmin: 4.786829060955809
-----------------------------------
expected_val_sq_zmax:		    2.829297433167023
expected_val_sq_zmin:		    2.771412217589978
upper bound for sq_zmax or sq_zmin: 4.861574832484699
-----------------------------------
expected_val_sq_zmax:		    2.8349831086565525
expected_val_sq_zmin:		    2.760512533413529
upper bo